In [1]:
import pandas as pd
import numpy as np

# 1. Cargar la base de datos
df = pd.read_json("/content/drive/MyDrive/nwinners_list.json")

# Nota: En Python 3 las cadenas ya están en Unicode, por lo que no es necesario convertirlas.

# 2. Reemplazar entradas vacías por NaN
df.replace("", np.nan, inplace=True)
df = df.infer_objects(copy=False)

# 3. Eliminar el asterisco que aparece en algunos nombres y eliminar espacios sobrantes
df['name'] = df['name'].str.replace(r'\*', '', regex=True).str.strip()

# 4. Eliminar individuos repetidos.
# Se considera duplicado si coinciden nombre, año y categoría (para conservar a aquellos que ganaron en años distintos)
df.drop_duplicates(subset=['name', 'year', 'category'], inplace=True)

# 5. Ordenar por nombre, country y year
df.sort_values(by=['name', 'country', 'year'], inplace=True)

# 6. Rellenar la categoría faltante.
# Se intenta extraer la categoría de la columna "text" buscando palabras clave conocidas.
def fill_category(row):
    if pd.isna(row['category']) or row['category'] == '':
        categorias = ['Physics', 'Chemistry', 'Physiology or Medicine', 'Literature', 'Peace', 'Economics']
        for cat in categorias:
            if cat in row['text']:
                return cat
        return np.nan
    else:
        return row['category']

df['category'] = df.apply(fill_category, axis=1)

# 7. Rellenar el género de los registros faltantes.
df['gender'] = df['gender'].fillna('Desconocido')

# 8. Convertir fecha de nacimiento y fecha de muerte a tipo NumPy datetime64
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors='coerce')
df['date_of_death'] = pd.to_datetime(df['date_of_death'], errors='coerce')

# 9. Eliminar premios que no fueron dados a personas.
# Por ejemplo, se filtran aquellos cuyo nombre contiene indicativos de organizaciones
df = df[~df['name'].str.contains("Committee|Friends", na=False)]

# Guardar la base de datos limpia en un nuevo archivo JSON
df.to_json("nwinners_list_clean.json", orient='records', date_format='iso')


<ipython-input-1-a0411c7e9e45>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("", np.nan, inplace=True)


In [5]:
df.head()

,link,name,year,category,country,born_in,text,date_of_birth,date_of_death,place_of_birth,place_of_death,gender
884,http://en.wikipedia.org/wiki/14th_Dalai_Lama,14th Dalai Lama,1989,Peace,India,NaN,"14th Dalai Lama , born in Taktser , China ,...",NaT,NaT,NaN,NaN,Desconocido
291,http://en.wikipedia.org/wiki/A._Michael_Spence,A. Michael Spence,2001,Economics,United States,NaN,"A. Michael Spence , Economics, 2001",NaT,NaT,NaN,NaN,Desconocido
1131,http://en.wikipedia.org/wiki/Aage_Bohr,Aage Bohr,1975,Physics,Denmark,NaN,"Aage Bohr , Physics, 1975",NaT,NaT,NaN,NaN,Desconocido
863,http://en.wikipedia.org/wiki/Aaron_Ciechanover,Aaron Ciechanover,2004,Chemistry,Israel,NaN,"Aaron Ciechanover , Chemistry, 2004",NaT,NaT,NaN,NaN,Desconocido
530,http://en.wikipedia.org/wiki/Aaron_Klug,Aaron Klug,1982,Chemistry,United Kingdom,NaN,"Aaron Klug , born in Lithuania , Chemistry, 1982",NaT,NaT,NaN,NaN,Desconocido


In [2]:
df1 = pd.read_json("/content/nwinners_list_clean.json")
df1.head()

,link,name,year,category,country,born_in,text,date_of_birth,date_of_death,place_of_birth,place_of_death,gender
0,http://en.wikipedia.org/wiki/14th_Dalai_Lama,14th Dalai Lama,1989,Peace,India,None,"14th Dalai Lama , born in Taktser , China ,...",NaN,NaN,NaN,NaN,Desconocido
1,http://en.wikipedia.org/wiki/A._Michael_Spence,A. Michael Spence,2001,Economics,United States,None,"A. Michael Spence , Economics, 2001",NaN,NaN,NaN,NaN,Desconocido
2,http://en.wikipedia.org/wiki/Aage_Bohr,Aage Bohr,1975,Physics,Denmark,None,"Aage Bohr , Physics, 1975",NaN,NaN,NaN,NaN,Desconocido
3,http://en.wikipedia.org/wiki/Aaron_Ciechanover,Aaron Ciechanover,2004,Chemistry,Israel,None,"Aaron Ciechanover , Chemistry, 2004",NaN,NaN,NaN,NaN,Desconocido
4,http://en.wikipedia.org/wiki/Aaron_Klug,Aaron Klug,1982,Chemistry,United Kingdom,None,"Aaron Klug , born in Lithuania , Chemistry, 1982",NaN,NaN,NaN,NaN,Desconocido
